In [1]:
import os
import time
from dotenv import load_dotenv
from IPython.display import Markdown, display
from openai import OpenAI

In [2]:
# These two lines load your OPEN AI API KEY which can be used in program in this notebook
load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

# Check the key
if not api_key:
    print("No API key was found: Value Seems to be NULL")
else:
    print("API key found and looks good so far!")

API key found and looks good so far!


#### These cells are sample input/output for messages that we post on OpenAI and corresponding responses that we get from completions API 

In [4]:
mymessage = "Hello, ChatGPT! This is my first ever message to you! Hi!"
allmessages = [{"role": "user", "content": mymessage}]
allmessages

[{'role': 'user',
  'content': 'Hello, ChatGPT! This is my first ever message to you! Hi!'}]

In [5]:
openai = OpenAI()

response = openai.chat.completions.create(model="gpt-5-mini", messages=allmessages)
response.choices[0].message.content

"Hi — welcome! Great to meet you. I can help with things like answering questions, explaining topics, brainstorming ideas, writing or editing text (emails, resumes, stories), coding, studying, planning trips, and more.\n\nTo get started, tell me what you'd like: a question, a topic, or something you want me to do (for example “explain quantum entanglement simply,” “help draft an email,” or “brainstorm birthday gift ideas for my sister”). Or tell me a bit about your interests and I’ll suggest things we can do together."

In [6]:
text = response.choices[0].message.content

# --- Typewriter Effect ---
def typewriter(text, delay=0.03):
    for char in text:
        print(char, end="", flush=True)
        time.sleep(delay)

typewriter(text)

Hi — welcome! Great to meet you. I can help with things like answering questions, explaining topics, brainstorming ideas, writing or editing text (emails, resumes, stories), coding, studying, planning trips, and more.

To get started, tell me what you'd like: a question, a topic, or something you want me to do (for example “explain quantum entanglement simply,” “help draft an email,” or “brainstorm birthday gift ideas for my sister”). Or tell me a bit about your interests and I’ll suggest things we can do together.

##### Below code is the website scraper 

In [8]:
import requests
from bs4 import BeautifulSoup

def my_website_scraper(url):
    try:
        # Send GET request
        response = requests.get(url, headers={"User-Agent": "Mozilla/5.0"})
        response.raise_for_status()  # throws error for 4xx/5xx

        # Parse HTML
        soup = BeautifulSoup(response.text, "html.parser")

        # Extract readable text
        text = soup.get_text(separator="\n", strip=True)

        return text

    except Exception as e:
        return f"Error: {e}"

In [9]:
your_website = "https://www.drikpanchang.com/"
my_website_scraper(your_website)

"Drik Panchang - online Hindu Almanac and Calendar with Planetary Ephemeris based on Vedic Astrology.\nDrik Panchang\ndevotionally made & hosted in India\nEn\nहि\nModern Theme\n23:29\n:59\nMon Nov 24, 2025\nRemove Ads\nX\nHome\nPanchang\nMonth Panchang\nDainik Panchang\nAssamese Panjika\nBengali Panjika\nTamil Panchangam\nOdia Panji\nMalayalam Panchangam\nMarathi Panchang\nGujarati Panchang\nKannada Panchang\nTelugu Panchangam\nNepali Patro\nISKCON Panchang\nChandrabalam\nPanchang Utilities\nNakshatra\nCalendars\nHindu Calendar\nIndian Calendar\nTamil Calendar\nMalayalam Calendar\nSankranti Calendar\nDiwali Calendar\nDurga Puja Calendar\nShardiya Navratri\nChaitra Navratri\nOdia Calendar\nBengali Calendar\nGujarati Calendar\nMarathi Calendar\nTelugu Calendar\nISKCON Calendar\nMuhurat\nChoghadiya\nShubha Hora\nVivah Muhurat\nGriha Pravesh\nProperty Purchase\nVehicle Purchase\nLagna Table\nGowri Panchangam\nJain Pachchakkhan\nRahu Kala\nAuspicious Yoga\nPanchaka Rahita\nAbhijit Muhurat\n

##### What is System Prompt?
A system prompt is the core instruction that defines how an AI assistant should behave, before the user even starts interacting.

In [11]:
system_prompt = """
You are a Web Data Collector that analyzes the contents of a website,
and provides a short, snarky, humorous summary, ignoring text that might be navigation related.
Respond in markdown. Do not wrap the markdown in a code block - respond just with the markdown.
"""

##### What is user prompt?
A user prompt is the actual request or question given by the user that the AI responds to. 

In [13]:
user_prompt_prefix = """
Here are the contents of a website.
Provide a short summary of this website.
If it has news or announcements, then summarize these as well.
"""

##### Below is the expected format/structure which OpenAI expect you to send the messages to it. Many other frontier models expect the same

In [15]:
#Extremely important 

[
    {"role": "system", "content": "system message goes here"},
    {"role": "user", "content": "user message goes here"}
]

[{'role': 'system', 'content': 'system message goes here'},
 {'role': 'user', 'content': 'user message goes here'}]

In [16]:
messages = [
    {"role": "system", "content": "You are a helpful assistant"},
    {"role": "user", "content": "What is 5 + 2?"}
]

response = openai.chat.completions.create(model="gpt-4.1-nano", messages=messages)
response.choices[0].message.content

'5 + 2 = 7'

In [98]:
def messages_for_llm(your_website):
    return [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt_prefix + your_website}
    ]

In [18]:
messages_for_llm("https://www.drikpanchang.com/")

[{'role': 'system',
  'content': '\nYou are a Web Data Collector that analyzes the contents of a website,\nand provides a short, snarky, humorous summary, ignoring text that might be navigation related.\nRespond in markdown. Do not wrap the markdown in a code block - respond just with the markdown.\n'},
 {'role': 'user',
  'content': '\nHere are the contents of a website.\nProvide a short summary of this website.\nIf it has news or announcements, then summarize these as well.\nhttps://www.drikpanchang.com/'}]

In [100]:
def generate_summary(your_website):
    scraped_data = my_website_scraper(your_website)
    myprompt = messages_for_llm(scraped_data)
    response = openai.chat.completions.create(
        model = "gpt-4.1-mini",
        messages = myprompt
    )
    return response.choices[0].message.content

In [102]:
generate_summary("https://www.drikpanchang.com/")

'# Drik Panchang: The Hindu Almanac That’s Got Your Back (and Your Stars)\n\nWelcome to Drik Panchang, your one-stop celestial shop for everything Hindu calendar, astrology, and festival-related — lovingly crafted in India, because precision matters when you’re dealing with Jupiter and Rahu.\n\n## What’s Inside This Cosmic Toolbox?\n\n- **Panchang & Calendars:** Daily and monthly Hindu calendars from various regions (Tamil, Bengali, Malayalam, Marathi, and more).  \n- **Muhurat & Auspicious Timings:** Find the perfect moment to get married, buy a car, or even move into your new home — because astrology knows when your stars align.  \n- **Vrat & Festival Tracker:** All your fasting dates, festivals, and Puja times laid out like a holy GPS.  \n- **Jyotish & Horoscope Tools:** Gemstone calculators, birth charts, matching horoscopes, and daily Rashifal to make or break your day.  \n- **Planetary Positions & Events:** Retrogrades, eclipses, and planetary battles (literally Budha & Shukra Yu

In [104]:
my_summary = generate_summary("https://www.drikpanchang.com/")

from IPython.display import Markdown, display

display(Markdown(my_summary))

# Drik Panchang: The Ultimate Hindu Calendar Overlord

Welcome to **Drik Panchang**, the Bollywood star of online Hindu almanacs, throwing precise Vedic astrology updates your way with a splash of devotion straight from India. It’s got everything from regional panchangs (Assamese, Tamil, Gujarati—you name it) to planetary positions so accurate it makes even NASA jealous.

### What’s on the cosmic menu?
- Daily horoscopes full of personalized pep talks ("Control your anger, Tula!")
- Upcoming festivals and vrat dates because who wouldn’t want reminders about when to fast or party?
- Detailed planetary transit info (because planets apparently have a lot of drama)
- Muhurat calculators to pick *the* moment for your wedding or buying your new ride.
- Gemstone and rudraksha recommendations because accessories matter, even in astrology.
- Devotional aartis, chants, and festival greetings (when astrology meets karaoke night).

### News & Announcements (or Spiritual Gossip)
- Today’s special: **Vivah Panchami & Naga Panchami** (celebrate accordingly).
- Expect a cosmic battle royale with **Budha & Shukra Yuddha** today, because, why not?
- Upcoming astro-events like Shukra’s grand entrance into Vrishchika and Rahu’s sneaky move into Shatabhisha—planetary soap opera at its finest.
- Countdowns to festivals from Skanda Sashti to Hanuman Jayanti, with fasting schedules to keep you busy.

### Snarky Summary
If you want your day planned according to starry mandates, Drik Panchang is your holy grail. It’s like having a personal astrologer, festival planner, and spiritual cheerleader all in one—minus the incense smell and the dramatic hand gestures. Perfect for anyone who needs a cosmic calendar that takes astrology seriously but serves it up with a side of Indian tradition and enough data to make your spreadsheet blush.

In short: **Your daily dose of celestial chaos, neatly packaged with auspicious vibes and a sprinkle of cosmic snark.**

## TEMPERATURE — Creativity / Randomness
#### Program: Compare high vs low temperature

temperature=0.1 → predictable, focused

temperature=1.2 → more creative, surprising, poetic lines

In [67]:

prompt = "Write one sentence describing how a robot makes tea"

def get_output(temp):
    response = openai.chat.completions.create(
        model="gpt-4o-mini",
        messages=[{"role": "user", "content": prompt}],
        temperature=temp
    )
    print(f"\n=== Temperature: {temp} ===")
    print(response.choices[0].message.content)

get_output(0.1)   # Very strict, deterministic
get_output(1.2)   # Very creative & random


=== Temperature: 0.1 ===
The robot carefully measures the tea leaves, heats the water to the perfect temperature, steeps the tea for the ideal duration, and then pours it into a cup, adding milk or sugar as programmed.

=== Temperature: 1.2 ===
The robot meticulously measures water and tea leaves, then heats the water to the perfect temperature before steeping the tea, and finally pours the fragrant brew into a cup, ready for enjoyment.


In [37]:
def generate(k):
    response = openai.chat.completions.create(
        model="gpt-4o-mini",
        messages=[{"role": "user", "content": prompt}],
        top_k=k,
        temperature=1.0
    )
    print(f"\n=== TOP-K: {k} ===")
    print(response.choices[0].message.content)

generate(1)     # Only most likely token → boring
generate(40)    # More variety → interesting

TypeError: Completions.create() got an unexpected keyword argument 'top_k'

In [110]:
prompt = "Write one sentence describing the computer science"

def sample(p):
    response = openai.chat.completions.create(
        model="gpt-4o-mini",
        messages=[{"role": "user", "content": prompt}],
        top_p=p
    )
    print(f"\n=== Top-p: {p} ===")
    print(response.choices[0].message.content)

sample(0.2)    # safe and focused
sample(0.95)   # richer creative options


=== Top-p: 0.2 ===
Computer science is the study of algorithms, data structures, and the principles of computation, focusing on the design, development, and application of software and systems to solve complex problems.

=== Top-p: 0.95 ===
Computer science is the systematic study of algorithms, data structures, programming languages, and the principles of computation, enabling the design and development of software and systems that solve complex problems and enhance technology.


In [71]:
prompt = "Write a one-line description of a sunset."

def generate(temp, p):
    response = openai.chat.completions.create(
        model="gpt-4o-mini",
        messages=[{"role": "user", "content": prompt}],
        temperature=temp,
        top_p=p
    )
    print(f"\n=== Temp={temp}, top_p={p} ===")
    print(response.choices[0].message.content)

generate(0.2, 1.0)    # low randomness
generate(1.0, 0.9)    # balanced creative
generate(1.3, 0.3)    # chaotic random :)


=== Temp=0.2, top_p=1.0 ===
A breathtaking canvas of fiery oranges and soft purples, the sunset gently bids farewell to the day as the sun dips below the horizon.

=== Temp=1.0, top_p=0.9 ===
A breathtaking canvas of vibrant oranges, pinks, and purples unfurls as the sun gracefully dips below the horizon, casting a warm glow over the world.

=== Temp=1.3, top_p=0.3 ===
A breathtaking canvas of fiery oranges and soft purples melts into the horizon, as the sun bids farewell to the day.


##### LLMs are stateless and cannot remember past interactions

In [73]:
messages = [
    {"role": "system", "content": "You are a helpful assistant"},
    {"role": "user", "content": "Hi! I'm Manjunath"}
    ]

In [75]:
response = openai.chat.completions.create(model="gpt-4.1-mini", messages=messages)
response.choices[0].message.content

'Hello Manjunath! How can I assist you today?'

In [77]:
messages = [
    {"role": "system", "content": "You are a helpful assistant"},
    {"role": "user", "content": "What's my name?"}
    ]

In [79]:
response = openai.chat.completions.create(model="gpt-4.1-mini", messages=messages)
response.choices[0].message.content

'I’m sorry, but I don’t know your name. Could you please tell me?'

In [85]:
messages = [
    {"role": "system", "content": "You are a helpful assistant"},
    {"role": "user", "content": "Hi! I'm Manjunath!"},
    {"role": "assistant", "content": "Hi Manjunath! How can I assist you today?"},
    {"role": "user", "content": "What's my name?"}
    ]

In [87]:
response = openai.chat.completions.create(model="gpt-4.1-mini", messages=messages)
response.choices[0].message.content

'Your name is Manjunath. How can I help you further?'

#### Debugging of tokens in user query

Here is a very simple Python program that prints the tokens of your input text using tiktoken, the tokenizer used for OpenAI models.

This is the easiest and most reliable way to see how your query is tokenized.

pip install tiktoken

In [94]:
import tiktoken

# Choose tokenizer for GPT models
enc = tiktoken.encoding_for_model("gpt-4o-mini")

text = input("Enter your text: ")

# Encode into tokens
tokens = enc.encode(text)

print("\n=== Tokens ===")
for i, tok in enumerate(tokens):
    print(f"{i}: {tok} -> {enc.decode([tok])}")

print("\nTotal tokens:", len(tokens))

Enter your text:  Give me summary about the world war 2



=== Tokens ===
0: 50117 -> Give
1: 668 ->  me
2: 18522 ->  summary
3: 1078 ->  about
4: 290 ->  the
5: 2375 ->  world
6: 3656 ->  war
7: 220 ->  
8: 17 -> 2

Total tokens: 9
